In [1]:
from helper_functions.get_processed_data import *
from model_training_functions.kfold_cross_validation import *
path = '/Users/barganor/Downloads/XY_train (1).csv'

data = pd.read_csv(path)

In [2]:
%%capture 
data = get_processed_data(data)

In [3]:
data.head()

,enrollment,target,PC-1,PC-2,PC-3
1,1,0,-0.187175,0.588019,-0.088033
4,0,1,-0.433266,-0.065953,0.230475
5,1,0,0.644192,-0.142224,0.014075
6,1,0,-0.218415,-0.107779,-0.113230
7,1,1,0.095160,0.169688,0.147305


In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
train_model_by_kfold(data, clf)

0.7070552147239264
0.6871165644171779
0.718558282208589
0.7062883435582822
0.7108895705521472
0.7078220858895705
0.7254601226993865
0.718558282208589
0.7116564417177914
0.7122026093630084


Cross-Validation accuracy: 0.711 +/- 0.010
